In [113]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import  FewShotPromptTemplate

from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat=ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
  llm=chat,
  max_token_limit=120,
  return_messages=True  #문자열로 바꾸지 말고 실제 메세지로 바꿔달라는 것을 의미
)

examples = [
    {
        "film": "Dune2",
        "answer": """
        Here is what I know:
        🏜️ 🪐 🦂 
        """,
    },
    {
        "film": "Top Gun",
        "answer": """
        I know this:
        🛩️ 👨‍✈️ 🔥
        """,
    },
    {
        "film": "The Godfather",
        "answer": """
        I know this:
        👨‍👨‍👦🔫🍝
        """,
    },
]
example_prompt = PromptTemplate.from_template(
  "Human: {film} \n AI:{answer}"
) 

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples = examples, 
  input_variables=["film"],
  suffix="Human:What are three emojis that come to mind when you look at the title of this {film}?"
)

def load_memory(_):
  return memory.load_memory_variables({})["history"]



chain = RunnablePassthrough.assign(history=load_memory)| prompt|chat


def invoke_chain(question):
  result = chain.invoke(
    {"film":question}
  )
  memory.save_context({"input":question}, {"output":result.content})
  print(question, "---", result, result.content)

invoke_chain("starwars")
invoke_chain("harry potter")

starwars --- content='AI: \n        Here are three emojis that come to mind when I think of Star Wars:\n        ⭐️🚀👽' AI: 
        Here are three emojis that come to mind when I think of Star Wars:
        ⭐️🚀👽
harry potter --- content='AI: ⚡🧙🏰' AI: ⚡🧙🏰


In [118]:
memory.load_memory_variables({"inputs":"starwars"})[history]


{'history': [HumanMessage(content='starwars'),
  AIMessage(content='AI: \n        Here are three emojis that come to mind when I think of Star Wars:\n        ⭐️🚀👽'),
  HumanMessage(content='harry potter'),
  AIMessage(content='AI: ⚡🧙🏰')]}

In [111]:
from langchain.memory import ConversationKGMemory
memory = ConversationKGMemory(
  llm=chat,
  return_messages=True
)

def add_message(input, output):
  memory.save_context({"input":input}, {"output":output})

add_message("Hi, I'm moon, I live in Seoul", "Wow that is so cool")
memory.load_memory_variables({"inputs":"Who is moon?"})

{'history': [SystemMessage(content='On moon: moon is Hi. moon lives in Seoul.')]}

In [121]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import  FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat=ChatOpenAI(temperature=0.1, streaming=True, 
                callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "film": "Dune2",
        "answer": """
        🏜️ 🪐 🦂 
        """,
    },
    {
        "film": "Top Gun",
        "answer": """
        🛩️ 👨‍✈️ 🔥
        """,
    },
    {
        "film": "The Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
]
example_prompt = PromptTemplate.from_template(
  "Human: I want to see three emojis that represent thus {film} \n AI: {film} : {answer}"
) 

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples = examples, 
  input_variables=["film"],
  suffix="Human:What are three emojis that come to mind when you look at the title of this {film}?"
)

chain = prompt | chat
result = chain.invoke({"film":"harry potter"})
print(result.content)

ValidationError: 2 validation errors for ChatOpenAI
callbacks -> 0
  instance of BaseCallbackHandler expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCallbackHandler)
callbacks
  instance of BaseCallbackManager expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCallbackManager)